In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

In [3]:
data = pd.read_csv(r'C:\Users\santi\OneDrive\Escritorio\python_project\bases_prueba\ALEGRIA.csv', thousands=',', decimal='.')

In [7]:
data['CANTIDAD DE EMPRESAS ']

CODIGO DANE  AÑO 
221          2017    557
             2018    570
             2019    588
             2020    563
             2021    620
                    ... 
99773        2018    297
             2019    315
             2020    294
             2021    349
             2022    395
Name: CANTIDAD DE EMPRESAS , Length: 6612, dtype: int64

In [4]:
import pandas as pd
from linearmodels.panel import PanelOLS
from sklearn.metrics import mean_squared_error, r2_score

# Asegúrate de que todas las columnas necesarias sean numéricas y que el índice esté configurado para panel data
# Usualmente, un índice jerárquico con entidad y tiempo, por ejemplo: ['CODIGO DANE', 'AÑO']

# Definir las variables independientes y dependiente
variables_independientes = [
    '(1) NRO_CORRESPONSALES_FÍSICOS_PROPIOS',
    '(2) NRO_CORRESPONSALES_FÍSICOS_TERCERIZADOS',
    '(3) NRO_CORRESPONSALES_FÍSICOS_ACTIVOS',
    '(5) NRO_DEPÓSITOS_CORRESPONSALES_FÍSICOS',
    '(6) MONTO_DEPÓSITOS_CORRESPONSALES_FÍSICOS',
    '(7) NRO_GIROS_ENVIADOS_CORRESPONSALES_FÍSICOS',
    '(8) MONTO_GIROS_ENVIADOS_CORRESPONSALES_FÍSICOS',
    '(12) MONTO_PAGOS_CORRESPONSALES_FÍSICOS',
    '(11) NRO_PAGOS_CORRESPONSALES_FÍSICOS',
    '(13) NRO_RETIROS_CORRESPONSALES_FÍSICOS',
    '(15) NRO_TRANSFERENCIAS_CORRESPONSALES_FÍSICOS',
    '(16) MONTO_TRANSFERENCIAS_CORRESPONSALES_FÍSICOS',
    '(26) SALDO_CTAS_AHORRO_ACTIVAS'
]

variable_dependiente = 'VALOR AGREGADO'

# Asegurarse de que el DataFrame tiene un MultiIndex adecuado para datos de panel
# El MultiIndex debería estar configurado como ['CODIGO DANE', 'AÑO'] o lo que corresponda en tu caso
data = data.set_index(['CODIGO DANE', 'AÑO'])

# Crear el DataFrame con las variables dependiente e independientes
X = data[variables_independientes]  # Variables independientes
y = data[variable_dependiente]  # Variable dependiente

# Añadir una constante al modelo (para la regresión OLS en panel)
X = X.copy()
X['const'] = 1  # Agrega una constante

# Crear y ajustar el modelo de datos en panel
modelo_panel = PanelOLS(y, X)

# Entrenar el modelo (ajustarlo a los datos)
resultado = modelo_panel.fit()

# Imprimir el resumen de los resultados
print(resultado)

# Evaluar el modelo con métricas como MSE y R2
y_pred = resultado.predict().fitted_values
mse = mean_squared_error(y, y_pred)
r2 = r2_score(y, y_pred)

print(f'Error cuadrático medio (MSE): {mse}')
print(f'Coeficiente de determinación (R²): {r2}')


ValueError: exog does not have full column rank. If you wish to proceed with model estimation irrespective of the numerical accuracy of coefficient estimates, you can set check_rank=False.

In [ ]:
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

resultados_por_region = {}

regiones = data['REGION'].unique()

for region in regiones:
    df_region = data[data['REGION'] == region]
    
    X = df_region[[
    '(1) NRO_CORRESPONSALES_FÍSICOS_PROPIOS',
    '(2) NRO_CORRESPONSALES_FÍSICOS_TERCERIZADOS',
    '(3) NRO_CORRESPONSALES_FÍSICOS_ACTIVOS',
    '(5) NRO_DEPÓSITOS_CORRESPONSALES_FÍSICOS',
    '(6) MONTO_DEPÓSITOS_CORRESPONSALES_FÍSICOS',
    '(7) NRO_GIROS_ENVIADOS_CORRESPONSALES_FÍSICOS',
    '(8) MONTO_GIROS_ENVIADOS_CORRESPONSALES_FÍSICOS',
    '(12) MONTO_PAGOS_CORRESPONSALES_FÍSICOS',
    '(11) NRO_PAGOS_CORRESPONSALES_FÍSICOS',
    '(13) NRO_RETIROS_CORRESPONSALES_FÍSICOS',
    '(15) NRO_TRANSFERENCIAS_CORRESPONSALES_FÍSICOS',
    '(16) MONTO_TRANSFERENCIAS_CORRESPONSALES_FÍSICOS',
    '(26) SALDO_CTAS_AHORRO_ACTIVAS'
]]  
    y = df_region['VALOR AGREGADO']
    
    X = sm.add_constant(X)

    modelo = sm.OLS(y, X).fit()
    valores_predichos = modelo.predict(X)

    residuos = modelo.resid
    sns.set(style='whitegrid')

    plt.figure(figsize=(10, 6))
    plt.scatter(valores_predichos, residuos, alpha=0.5)
    plt.axhline(y=0, color='red', linestyle='--')  
    plt.title('Gráfico de Dispersión: Residuos vs. Valores Predichos')
    plt.xlabel('Valores Predichos')
    plt.ylabel('Residuos')
    plt.xlim([valores_predichos.min(), valores_predichos.max()])  
    plt.ylim([residuos.min(), residuos.max()])  
    plt.show()
    
    resultados_por_region[region] = modelo.summary()


for region, resultados in resultados_por_region.items():
    display(f"Resultados para {region}:")
    display(resultados)
    display("\n" + "="*50 + "\n")


Aquí tienes un resumen más conciso de los resultados de tu regresión OLS:

### Resumen de Resultados

1. **Variable Dependiente**: `VALOR AGREGADO`.
2. **R-squared**: **0.906** - El modelo explica el 90.6% de la variabilidad en el `VALOR AGREGADO`.
3. **Adj. R-squared**: **0.905** - Buen ajuste del modelo considerando el número de variables.
4. **F-statistic**: **862.1** (p < 0.0001) - Indica que el modelo es significativo.
5. **Número de Observaciones**: **1176** - Tamaño de muestra grande para mayor precisión.

### Coeficientes Significativos

- **Intercepción**: 193.26.
- **NRO_CORRESPONSALES_FÍSICOS_PROPIOS**: +9.78 - Aumento en el `VALOR AGREGADO` por cada unidad adicional.
- **NRO_DEPÓSITOS_CORRESPONSALES_FÍSICOS**: -0.019 - Disminución en el `VALOR AGREGADO` por cada depósito adicional.

### Notas sobre los Supuestos

- **Normalidad de Residuos**: Valores p bajos (Omnibus y Jarque-Bera) sugieren que los residuos no son normalmente distribuidos.
- **Multicolinealidad**: Alto número de condición (6.89e+11) indica posibles problemas entre variables independientes.

### Conclusión

El modelo tiene un buen ajuste y varias variables son significativas, pero se deben considerar problemas potenciales de normalidad y multicolinealidad en los residuos.

In [ ]:
class Graph:
    def __init__(self, vertices):
        self.V = vertices  # Número de vértices
        self.graph = [[0] * vertices for _ in range(vertices)]  # Matriz de capacidad

    def add_edge(self, u, v, w):
        self.graph[u][v] = w  # Añadir capacidad al arco u-v

    def bfs(self, s, t, parent):
        visited = [False] * self.V
        queue = [s]
        visited[s] = True

        while queue:
            u = queue.pop(0)

            for v in range(self.V):
                if not visited[v] and self.graph[u][v] > 0:  # Si hay capacidad disponible
                    queue.append(v)
                    visited[v] = True
                    parent[v] = u

                    if v == t:  # Si llegamos al sumidero
                        return True
        return False

    def ford_fulkerson(self, source, sink):
        parent = [-1] * self.V  # Array para almacenar el camino aumentado
        max_flow = 0  # Flujo máximo

        # Aumentar el flujo mientras haya un camino aumentado
        while self.bfs(source, sink, parent):
            # Encontrar la capacidad del flujo mínimo en el camino aumentado
            path_flow = float('Inf')
            s = sink
            while s != source:
                path_flow = min(path_flow, self.graph[parent[s]][s])
                s = parent[s]

            # Actualizar las capacidades de los arcos y arcos inversos
            v = sink
            while v != source:
                u = parent[v]
                self.graph[u][v] -= path_flow
                self.graph[v][u] += path_flow
                v = parent[v]

            max_flow += path_flow  # Sumar el flujo del camino aumentado

        return max_flow

# Ejemplo de uso
if __name__ == "__main__":
    g = Graph(6)  # Crear un grafo con 6 vértices
    g.add_edge(0, 1, 16)
    g.add_edge(0, 2, 13)
    g.add_edge(1, 2, 10)
    g.add_edge(1, 3, 12)
    g.add_edge(2, 1, 4)
    g.add_edge(2, 4, 14)
    g.add_edge(3, 2, 9)
    g.add_edge(3, 5, 20)
    g.add_edge(4, 3, 7)
    g.add_edge(4, 5, 4)

    source = 0  # Nodo fuente
    sink = 5    # Nodo sumidero

    print(f"El flujo máximo es: {g.ford_fulkerson(source, sink)}")
